# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

# Spark session

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("kmeans project").getOrCreate()

23/08/18 21:21:35 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.1.231 instead (on interface wlo1)
23/08/18 21:21:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/18 21:21:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/18 21:21:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Read the data

In [4]:
df = spark.read.csv("hack_data.csv",inferSchema=True, header=True)

In [5]:
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [8]:
df = df.drop("Location")

In [9]:
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|            70.8|
|                   20.0|            408.5|              0|             3.57|            8.0|           71.28|
|                    1.0|           390.69|              1|             2.79|            9.0|           71.57|
|

In [10]:
df.describe().show()

23/08/18 21:23:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|                    334|               334|               334|              334|               334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|              40.0|
|    max|                   60.0|            1330.5|                 1|             10.0|              1

# Preprocessing

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=df.columns, outputCol='features')

In [13]:
final_data = assembler.transform(df)

In [14]:
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|
|                    2.0|           228.08|              1|             2.48|            8.0|            70.8|[2.0,228.08,1.0,2...|
|                   20.0|            408.5|              0|             3.57

# Creating models for the results 

In [15]:
from pyspark.ml.clustering import KMeans

In [16]:
kmeans_two_hackers = KMeans(featuresCol='features',k=2)
model_two_hackers = kmeans_two_hackers.fit(final_data)

In [17]:
kmeans_three_hackers = KMeans(featuresCol='features',k=3)
model_three_hackers = kmeans_three_hackers.fit(final_data)

In [18]:
wssse_two_hackers = model_two_hackers.summary.trainingCost
wssse_three_hackers = model_three_hackers.summary.trainingCost

In [19]:
print("TWO HACKERS   : Within Set Sum of Squared Errors = " + str(wssse_two_hackers))
print("THREE HACKERS : Within Set Sum of Squared Errors = " + str(wssse_three_hackers))

TWO HACKERS   : Within Set Sum of Squared Errors = 6915193.171893844
THREE HACKERS : Within Set Sum of Squared Errors = 4008693.712472261


<span style="color:green">We can see that the WSSSE for THREE HACKERS is way less than the WSSSE for TWO HACKERS</span>

- We can then for now that the number of hackers is Three

- But lets see the distributions of hackers for both models, because it is the key element !!

In [20]:
clustered_data_two = model_two_hackers.transform(final_data)
clustered_data_three = model_three_hackers.transform(final_data)

In [25]:
clustered_count_two = clustered_data_two.groupBy("prediction").count()
clustered_count_three = clustered_data_three.groupBy("prediction").count()

In [26]:
clustered_count_two.show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  170|
|         0|  164|
+----------+-----+



In [27]:
clustered_count_three.show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   92|
|         2|  118|
|         0|  124|
+----------+-----+



<span style="color:green">Since the key factor is the distribution of the hacks, then the number of hackers is **2**</span>